In [1]:
import numpy as np
from enum import Enum
import queue

In [2]:
class Vertice:
    def __init__(self, rotulo):
        self.rotulo = rotulo

In [3]:
class Aresta:
    def __init__(self, inicio, fim):
        self.inicio = inicio
        self.fim = fim

    def e_extremidade(self, v):
        return self.inicio == v or self.fim == v

    def e_loop(self):
        return self.inicio == self.fim

    def oposto(self, v):
        if (self.inicio == v):
            return self.fim
        elif (self.fim == v):
            return self.inicio
        else:
            return None

In [4]:
class Cor(Enum):
    BRANCO = 1
    CINZENTO = 2
    PRETO = 3

In [5]:
class DadosBuscaLargura:
    def __init__(self, cor, d, pai):
        self.cor = cor
        self.d = d
        self.pai = pai

In [6]:
class DadosBuscaProfundidade:
    def __init__(self, cor, pai, d, f):
        self.cor = cor
        self.d = d
        self.pai = pai
        self.f = f

In [7]:
class Grafo:
    def __init__(self, n):
        self.vertices = [ Vertice(str(i + 1)) for i in range(n) ]
        self.arestas = []

    def adicionarVertice(self, v):
        self.vertices.append(v)    

    def adicionarAresta(self, origem, destino):
        self.arestas.append( Aresta(origem, destino) )

    def removerAresta(self, a):
        self.arestas.remove(a)

    def removerArestas(self, A):
        for a in A:
            self.removerAresta(a)

    def arestasIncidentes(self, v):
        return [ a for a in self.arestas if a.e_extremidade(v) ]

    def removerVertice(self, v):
        #Remove o vértice do vetor
        self.vertices.remove(v)
        
        #Remover as arestas ligadas ao vértice
        self.removerArestas(self.arestasIncidentes(v))    

    def numeroVertices(self):
        return len(self.vertices)

    def numeroArestas(self):
        return len(self.arestas)

    def matrizAdjacencia(self):
        n = self.numeroVertices()

        A = np.zeros((n, n), int)

        posicao = {}

        for (i, v) in enumerate(self.vertices):
            posicao[v] = i

        for a in self.arestas:
            vi = a.inicio
            vf = a.fim

            pi = posicao[vi]
            pf = posicao[vf]

            if (pi == pf):
                A[pi][pf] += 1
            else:
                A[pi][pf] += 1
                A[pf][pi] += 1

        return A

    def matrizIncidencia(self):
        n = self.numeroVertices()

        m = self.numeroArestas()

        S = np.zeros((n, m), int)

        posicao = {}

        for (i, v) in enumerate(self.vertices):
            posicao[v] = i

        for (j, a) in enumerate(self.arestas):
            vi = a.inicio
            vf = a.fim

            pi = posicao[vi]
            pf = posicao[vf]

            if (pi == pf):
                S[pi][j] = 2
            else:
                S[pi][j] = 1
                S[pf][j] = 1

        return S

    def listaAdjacencia(self):
        L = {}

        for v in self.vertices:
            L[v] = []

        for e in self.arestas:
            if e.inicio != e.fim:
                L[e.inicio].append(e.fim)            
            L[e.fim].append(e.inicio)
            
        return L

    def grau(self, v):
        #Define o grau inicial
        c = 0

        for e in self.arestas:
            if e.e_extremidade(v):
                if e.e_loop():
                    c += 2
                else:
                    c += 1

        return c

    def graus(self):
        n = self.numeroVertices()
        
        d = [0] * n

        posicao = dict( [ (v, i) for (i, v) in enumerate(self.vertices)] )

        for e in self.arestas:
            pi = posicao[e.inicio]
            pf = posicao[e.fim]

            d[pi] += 1
            d[pf] += 1

        return d

    def grau_medio(self):
        ds = self.graus()

        S = 0

        for d in ds:
            S += d

        return S / self.numeroVertices()

    def adjacentes(self, v):
        adj = []
        
        for e in self.arestas:
            if (e.e_extremidade(v)):
                adj.append(e.oposto(v))

        return adj

    def busca_largura(self, s):
        mapa = dict()
        
        for u in self.vertices:
            if (u != s):
                mapa[u] = DadosBuscaLargura(Cor.BRANCO, float('inf'), None)

        mapa[s] = DadosBuscaLargura(Cor.CINZENTO, 0, None)

        Q = queue.Queue()
        Q.put(s)
        while not Q.empty():
            u = Q.get()
            um = mapa[u]
            
            for v in self.adjacentes(u):
                vm = mapa[v]
                
                if vm.cor == Cor.BRANCO:
                    vm.cor = Cor.CINZENTO
                    vm.d = um.d + 1
                    vm.pai = u

                    Q.put(v)
                    
            um.cor = Cor.PRETO

        return mapa
    
    def busca_profundidade(self):
        mapa = dict()
        
        for u in self.vertices:
            mapa[u] = DadosBuscaProfundidade(Cor.BRANCO, None, float('inf'), float('inf'))
            
        tempo = 0
        
        for u in self.vertices:
            if mapa[u].cor == Cor.BRANCO:
                    tempo = self.busca_profundidade_vertice(u, mapa, tempo)
                
        return mapa
    
    def busca_profundidade_vertice(self, u, mapa, tempo):
        tempo += 1
        
        mapa[u].d = tempo
        mapa[u].cor = Cor.CINZENTO
        
        for v in self.adjacentes(u):
            if mapa[v].cor == Cor.BRANCO:
                mapa[v].pai = u
                tempo = self.busca_profundidade_vertice(v, mapa, tempo)
                
                
        mapa[u].cor = Cor.PRETO
        tempo += 1
        mapa[u].f = tempo
        
        return tempo

In [8]:
g = Grafo(5)

In [9]:
v1, v2, v3, v4, v5 = g.vertices[0], g.vertices[1], g.vertices[2], g.vertices[3], g.vertices[4]

In [10]:
g.numeroVertices()

5

In [11]:
g.adicionarAresta(v1, v2)
g.adicionarAresta(v1, v2)
g.adicionarAresta(v2, v3)
g.adicionarAresta(v1, v4)
g.adicionarAresta(v3, v4)
g.adicionarAresta(v3, v4)
g.adicionarAresta(v1, v1)

In [12]:
g.numeroArestas()

7

In [13]:
g.matrizAdjacencia()

array([[1, 2, 0, 1, 0],
       [2, 0, 1, 0, 0],
       [0, 1, 0, 2, 0],
       [1, 0, 2, 0, 0],
       [0, 0, 0, 0, 0]])

In [14]:
g.matrizIncidencia()

array([[1, 1, 0, 1, 0, 0, 2],
       [1, 1, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 1, 1, 1, 0],
       [0, 0, 0, 0, 0, 0, 0]])

In [15]:
l = g.listaAdjacencia()

In [16]:
for c in l.keys():
    V = l[c]
    
    print(c.rotulo, ": ", end="")
    for v in V:
        print(v.rotulo, " ", end="")
    print()

1 : 2  2  4  1  
2 : 1  1  3  
3 : 2  4  4  
4 : 1  3  3  
5 : 


In [17]:
g.grau(v1)

5

In [18]:
g.grau(v2)

3

In [19]:
g.grau(v3)

3

In [20]:
g.grau(v4)

3

In [21]:
g.graus()

[5, 3, 3, 3, 0]

In [22]:
g.grau_medio()

2.8

In [23]:
mapa = g.busca_largura(v1);

In [24]:
mapa[v1].pai == None

True

In [25]:
mapa[v1].d == 0

True

In [26]:
mapa[v2].pai == v1

True

In [27]:
mapa[v2].d == 1

True

In [28]:
mapa[v3].pai == v2

True

In [29]:
mapa[v3].d == 2

True

In [30]:
mapa[v4].pai == v1

True

In [31]:
mapa[v4].d == 1

True

In [32]:
mapa[v5].pai == None

True

In [33]:
mapa[v5].d

inf

In [34]:
mapa[v5].cor

<Cor.BRANCO: 1>

In [35]:
mapa = g.busca_profundidade()

In [36]:
mapa[v1].d

1

In [37]:
mapa[v1].f

8

In [39]:
mapa[v1].pai

In [40]:
mapa[v2].d

2

In [41]:
mapa[v2].f

7

In [43]:
mapa[v2].pai == v1

True